In [1]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath("../src"))
from core.Feature_engineering import FeatureEngineering
from core.woe_transformer import WoeTransformer
from core.proxy_target import create_proxy_target

In [2]:
# load our data
data = pd.read_csv("../data/processed/cleaned_data.csv")

In [3]:

# inspect our data
data

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1000.0,1000,2019-02-13 09:54:09+00:00,2,0
95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2019-02-13 09:54:25+00:00,2,0
95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2019-02-13 09:54:35+00:00,2,0
95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,ChannelId_3,3000.0,3000,2019-02-13 10:01:10+00:00,2,0


In [4]:
# check datat types
data.dtypes

TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype: object

In [5]:
#check unique enteries
data.nunique()

TransactionId           95662
BatchId                 94809
AccountId                3633
SubscriptionId           3627
CustomerId               3742
CurrencyCode                1
CountryCode                 1
ProviderId                  6
ProductId                  23
ProductCategory             9
ChannelId                   4
Amount                   1676
Value                    1517
TransactionStartTime    94556
PricingStrategy             4
FraudResult                 2
dtype: int64

In [6]:
fe_temp = FeatureEngineering([], [], []) # dummy(temporary) columns to instatntiate object for now
df_raw = fe_temp.extract_time_features(data) # extract time features hours,dayc etc from TransactionStartTime

2025-07-03 02:18:31,288 - core.Feature_engineering - INFO - Initializing FeatureEngineering with numeric features [] and categorical features []
2025-07-03 02:18:31,289 - core.Feature_engineering - INFO - Extracting datetime features...
2025-07-03 02:18:31,482 - core.Feature_engineering - INFO - Datetime features added: Hour, Day, Month, Year.


In [7]:
df_agg = fe_temp.aggregate_features(df_raw)  # aggregate features from extracted time features and the columns like amount value etc 
# inspect df_agg
df_agg # Data is aggregated per each customer in the dataset

2025-07-03 02:18:31,498 - core.Feature_engineering - INFO - Aggregating features to customer level
2025-07-03 02:18:39,682 - core.Feature_engineering - INFO - Aggregation complete — 3742 customers


,CustomerId,NetSpend,GrossVolume,TxnCount,AvgTxnValue,StdTxnValue,NumUniqueProducts,NumUniqueCategories,NumUniqueChannels,PreferredProvider,FraudCount,PreferredHour,MostCommonPricingStrategy,PreferredChannel,Recency,PreferredDayOfWeek
0,CustomerId_1,-10000.0,10000,1,10000.000000,0.000000,1,1,1,ProviderId_4,0,16,4,ChannelId_2,84,2
1,CustomerId_10,-10000.0,10000,1,10000.000000,0.000000,1,1,1,ProviderId_4,0,16,4,ChannelId_2,84,2
2,CustomerId_1001,20000.0,30400,5,6080.000000,4100.243895,3,2,2,ProviderId_4,0,8,2,ChannelId_3,90,4
3,CustomerId_1002,4225.0,4775,11,434.090909,518.805446,3,2,2,ProviderId_4,0,14,2,ChannelId_2,26,3
4,CustomerId_1003,20000.0,32000,6,5333.333333,3945.461528,4,2,2,ProviderId_6,0,14,2,ChannelId_3,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,CustomerId_992,20000.0,32000,6,5333.333333,4033.195590,3,2,2,ProviderId_6,0,10,2,ChannelId_3,5,4
3738,CustomerId_993,20000.0,32000,5,6400.000000,3781.534080,3,2,2,ProviderId_4,0,15,2,ChannelId_3,26,4
3739,CustomerId_994,543873.0,614077,101,6079.970297,14537.733039,6,3,2,ProviderId_6,0,17,2,ChannelId_3,1,4
3740,CustomerId_996,139000.0,151000,17,8882.352941,2619.216317,3,2,2,ProviderId_1,0,14,2,ChannelId_3,68,4


In [8]:
# Add proxy target to aggregated dataset
df_out = create_proxy_target(df_agg) # create proxy target and add it to aggregated dataset

print(f"The shape of our dataset with number of customers and features repsctively is: {df_out.shape}")
df_out[df_out["is_high_risk"] == 1].head(10) # inspect high risk customers

2025-07-03 02:18:39,717 - core.proxy_target - INFO - Starting RFM clustering for proxy target...
2025-07-03 02:18:44,508 - core.proxy_target - INFO - Cluster centers:
[[ 1.10663164e+00 -1.84892189e-01 -6.02426971e-02]
 [-9.07697835e-02  4.19477521e+01  3.85366828e+01]
 [-6.99571763e-01  9.85868364e-02  2.12718086e-02]]
2025-07-03 02:18:44,512 - core.proxy_target - INFO - Inversed centroids:
     Recency     TxnCount   GrossVolume
0  61.467909     7.645273  8.951265e+04
1  29.000000  4091.000000  1.049000e+08
2  12.492147    35.119110  3.108664e+05
2025-07-03 02:18:44,517 - core.proxy_target - INFO - Identified risky cluster: 0
2025-07-03 02:18:44,520 - core.proxy_target - INFO - Proxy target created: is_high_risk ✅


The shape of our dataset with number of customers and features repsctively is: (3742, 17)


,CustomerId,NetSpend,GrossVolume,TxnCount,AvgTxnValue,StdTxnValue,NumUniqueProducts,NumUniqueCategories,NumUniqueChannels,PreferredProvider,FraudCount,PreferredHour,MostCommonPricingStrategy,PreferredChannel,Recency,PreferredDayOfWeek,is_high_risk
0,CustomerId_1,-10000.0,10000,1,10000.000000,0.000000,1,1,1,ProviderId_4,0,16,4,ChannelId_2,84,2,1
1,CustomerId_10,-10000.0,10000,1,10000.000000,0.000000,1,1,1,ProviderId_4,0,16,4,ChannelId_2,84,2,1
2,CustomerId_1001,20000.0,30400,5,6080.000000,4100.243895,3,2,2,ProviderId_4,0,8,2,ChannelId_3,90,4,1
5,CustomerId_1004,2000.0,2000,1,2000.000000,0.000000,1,1,1,ProviderId_1,0,18,4,ChannelId_3,53,5,1
6,CustomerId_1005,48800.0,61200,9,6800.000000,4106.093034,3,2,2,ProviderId_4,0,8,2,ChannelId_3,47,4,1
7,CustomerId_1006,1000.0,1000,1,1000.000000,0.000000,1,1,1,ProviderId_5,0,7,2,ChannelId_3,65,1,1
8,CustomerId_1007,28000.0,28000,1,28000.000000,0.000000,1,1,1,ProviderId_5,0,6,2,ChannelId_3,82,5,1
9,CustomerId_1008,20000.0,20000,2,10000.000000,0.000000,1,1,1,ProviderId_3,0,13,2,ChannelId_3,66,6,1
11,CustomerId_1010,26335.0,31975,7,4567.857143,4887.057752,4,3,2,ProviderId_4,0,8,2,ChannelId_3,42,4,1
13,CustomerId_1014,763880.0,763880,8,95485.000000,119332.843401,1,1,1,ProviderId_3,0,9,2,ChannelId_3,50,1,1


In [9]:
df_agg["is_high_risk"].dtype

dtype('int64')

In [10]:
# Define features to perform Information Value analysis
agg_features = [
    'NetSpend', 'GrossVolume', 'TxnCount', 'AvgTxnValue',
       'StdTxnValue', 'NumUniqueProducts', 'NumUniqueCategories',
       'NumUniqueChannels', 'PreferredProvider', 'FraudCount', 'PreferredHour',
       'MostCommonPricingStrategy', 'PreferredChannel', 'Recency',
       'PreferredDayOfWeek',  
       ]

y = df_agg['is_high_risk'] 
X = df_agg.drop('is_high_risk', axis=1)

woe_tr = WoeTransformer(features=agg_features, bins=10)
woe_tr.fit(X, y)
print("IV scores:", woe_tr.get_iv())
iv_scores = woe_tr.get_iv()


c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future

IV scores: {'NetSpend': np.float64(0.4466034003310123), 'GrossVolume': np.float64(0.5380442530329408), 'TxnCount': np.float64(0.7882086919202942), 'AvgTxnValue': np.float64(0.07487638712587646), 'StdTxnValue': np.float64(0.34012624753576587), 'NumUniqueProducts': np.float64(0.580382990018795), 'NumUniqueCategories': np.float64(0.2168843939083898), 'NumUniqueChannels': np.float64(0.383844849193576), 'PreferredProvider': np.float64(0.0773455970459473), 'FraudCount': np.float64(0.008346538986988587), 'PreferredHour': np.float64(0.00010457055162813211), 'MostCommonPricingStrategy': np.float64(0.13335576506231642), 'PreferredChannel': np.float64(0.03920329180438436), 'Recency': np.float64(6.11744211709809), 'PreferredDayOfWeek': np.float64(0.06932630334051906)}


c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future

In [11]:
#   Convert to DataFrame for easy inspection
iv_df = pd.DataFrame(list(iv_scores.items()), columns=['Feature', 'IV'])

#  Set your threshold (example: IV >= 0.02 is usually 'weakly predictive')
iv_threshold = 0.02

#  Select only features above the threshold
selected_features = iv_df[iv_df['IV'] >= iv_threshold]['Feature'].tolist()

print("Selected features with IV >= 0.02:")
print(selected_features)

Selected features with IV >= 0.02:
['NetSpend', 'GrossVolume', 'TxnCount', 'AvgTxnValue', 'StdTxnValue', 'NumUniqueProducts', 'NumUniqueCategories', 'NumUniqueChannels', 'PreferredProvider', 'MostCommonPricingStrategy', 'PreferredChannel', 'Recency', 'PreferredDayOfWeek']


In [24]:
df_agg.to_csv("../data/raw/X_raw_withrisk.csv", index=False)
# Target variable
y = df_agg['is_high_risk']

# Final input X
X = df_agg[selected_features].copy()
X

,NetSpend,GrossVolume,TxnCount,AvgTxnValue,StdTxnValue,NumUniqueProducts,NumUniqueCategories,NumUniqueChannels,PreferredProvider,MostCommonPricingStrategy,PreferredChannel,Recency,PreferredDayOfWeek
0,-10000.0,10000,1,10000.000000,0.000000,1,1,1,ProviderId_4,4,ChannelId_2,84,2
1,-10000.0,10000,1,10000.000000,0.000000,1,1,1,ProviderId_4,4,ChannelId_2,84,2
2,20000.0,30400,5,6080.000000,4100.243895,3,2,2,ProviderId_4,2,ChannelId_3,90,4
3,4225.0,4775,11,434.090909,518.805446,3,2,2,ProviderId_4,2,ChannelId_2,26,3
4,20000.0,32000,6,5333.333333,3945.461528,4,2,2,ProviderId_6,2,ChannelId_3,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,20000.0,32000,6,5333.333333,4033.195590,3,2,2,ProviderId_6,2,ChannelId_3,5,4
3738,20000.0,32000,5,6400.000000,3781.534080,3,2,2,ProviderId_4,2,ChannelId_3,26,4
3739,543873.0,614077,101,6079.970297,14537.733039,6,3,2,ProviderId_6,2,ChannelId_3,1,4
3740,139000.0,151000,17,8882.352941,2619.216317,3,2,2,ProviderId_1,2,ChannelId_3,68,4


In [13]:
# Choose features for WoE
woe_features = [
    'NetSpend',
    'GrossVolume',
    'TxnCount',
    'AvgTxnValue',
    'Recency',
    'PreferredProvider',
    'MostCommonPricingStrategy',
    'PreferredChannel'
]

In [ ]:
import joblib
woe_tr = WoeTransformer(features=woe_features, bins=5)
woe_tr.fit(X, y)


# Inspect IV scores (optional)
print("IV scores:", woe_tr.get_iv())

# Replace raw columns with WoE versions
X_woe = woe_tr.transform(X)
X_woe

c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future

IV scores: {'NetSpend': np.float64(0.4466034003310123), 'GrossVolume': np.float64(0.5380442530329408), 'TxnCount': np.float64(0.7882086919202942), 'AvgTxnValue': np.float64(0.07487638712587646), 'Recency': np.float64(6.11744211709809), 'PreferredProvider': np.float64(0.0773455970459473), 'MostCommonPricingStrategy': np.float64(0.13335576506231642), 'PreferredChannel': np.float64(0.03920329180438436)}


c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_woe.py:187: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.fillna('NA')
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\_woe.py:310: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  outX[new_column_name] = tempX.replace(self.woe_bins[original_column_name])
c:\Users\adoni\Desktop\KAIM COURSE\WEEK-5\CreditRiskProbabilityModel\.venv\Lib\site-packages\xverse\transformer\

,StdTxnValue,NumUniqueProducts,NumUniqueCategories,NumUniqueChannels,PreferredDayOfWeek,NetSpend_woe,GrossVolume_woe,TxnCount_woe,AvgTxnValue_woe,Recency_woe,PreferredProvider_woe,MostCommonPricingStrategy_woe,PreferredChannel_woe
0,0.000000,1,1,1,2,0.755457,0.864950,0.897448,0.012640,6.892332,-0.035129,0.573687,-0.193549
1,0.000000,1,1,1,2,0.755457,0.864950,0.897448,0.012640,6.892332,-0.035129,0.573687,-0.193549
2,4100.243895,3,2,2,4,-0.006152,-0.072222,0.011104,-0.346197,6.892332,-0.035129,-0.119200,0.103140
3,518.805446,3,2,2,3,0.755457,0.864950,0.011104,0.320632,-0.981615,-0.035129,-0.119200,-0.193549
4,3945.461528,4,2,2,4,-0.006152,-0.072222,0.011104,-0.346197,0.000000,0.284038,-0.119200,0.103140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,4033.195590,3,2,2,4,-0.006152,-0.072222,0.011104,-0.346197,0.000000,0.284038,-0.119200,0.103140
3738,3781.534080,3,2,2,4,-0.006152,-0.072222,0.011104,-0.346197,-0.981615,-0.035129,-0.119200,0.103140
3739,14537.733039,6,3,2,4,-0.952163,-0.995837,-1.413437,-0.346197,0.000000,0.284038,-0.119200,0.103140
3740,2619.216317,3,2,2,4,-0.952163,-0.995837,-1.413437,0.012640,6.892332,-0.132094,-0.119200,0.103140


In [16]:
woe_cols = [col for col in X_woe.columns if '_woe' in col]
nonwoe_cols = [col for col in X_woe.columns if '_woe' not in col]
print(nonwoe_cols)

['StdTxnValue', 'NumUniqueProducts', 'NumUniqueCategories', 'NumUniqueChannels', 'PreferredDayOfWeek']


In [17]:
# Init the feature engineering for leftover numerics only
fe = FeatureEngineering(
    num_features=nonwoe_cols,  # only leftover numerics
    cat_features=[],
)

2025-07-03 02:21:29,340 - core.Feature_engineering - INFO - Initializing FeatureEngineering with numeric features ['StdTxnValue', 'NumUniqueProducts', 'NumUniqueCategories', 'NumUniqueChannels', 'PreferredDayOfWeek'] and categorical features []


In [18]:
#Build and fit-transform the numeric pipeline
fe.build_woe_pipeline()
X_nonwoe_scaled = fe.woe_pipeline.fit_transform(X_woe)

# Combine scaled non-WoE + original WoE columns
import scipy
from scipy import sparse

X_logit_final = sparse.hstack([
    X_nonwoe_scaled,
    sparse.csr_matrix(X_woe[woe_cols].values)
])

print(f"Shape of final logistic regression input: {X_logit_final.shape}")
X_logit_final

Shape of final logistic regression input: (3742, 13)


<COOrdinate sparse matrix of dtype 'float64'
	with 47242 stored elements and shape (3742, 13)>

In [19]:
dense_array = X_logit_final.toarray()
X_logit_final = pd.DataFrame(dense_array, columns= X_woe.columns)
X_logit_final

,StdTxnValue,NumUniqueProducts,NumUniqueCategories,NumUniqueChannels,PreferredDayOfWeek,NetSpend_woe,GrossVolume_woe,TxnCount_woe,AvgTxnValue_woe,Recency_woe,PreferredProvider_woe,MostCommonPricingStrategy_woe,PreferredChannel_woe
0,-0.131508,-1.115175,-1.153977,-1.404749,-0.606251,0.755457,0.864950,0.897448,0.012640,6.892332,-0.035129,0.573687,-0.193549
1,-0.131508,-1.115175,-1.153977,-1.404749,-0.606251,0.755457,0.864950,0.897448,0.012640,6.892332,-0.035129,0.573687,-0.193549
2,-0.089197,-0.135831,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,6.892332,-0.035129,-0.119200,0.103140
3,-0.126154,-0.135831,-0.114953,0.450075,-0.037397,0.755457,0.864950,0.011104,0.320632,-0.981615,-0.035129,-0.119200,-0.193549
4,-0.090795,0.353841,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,0.000000,0.284038,-0.119200,0.103140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,-0.089889,-0.135831,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,0.000000,0.284038,-0.119200,0.103140
3738,-0.092486,-0.135831,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,-0.981615,-0.035129,-0.119200,0.103140
3739,0.018508,1.333185,0.924070,0.450075,0.531458,-0.952163,-0.995837,-1.413437,-0.346197,0.000000,0.284038,-0.119200,0.103140
3740,-0.104480,-0.135831,-0.114953,0.450075,0.531458,-0.952163,-0.995837,-1.413437,0.012640,6.892332,-0.132094,-0.119200,0.103140


In [22]:
from utils.preprocessing_bundle import PreprocessingBundle

bundle = PreprocessingBundle(woe_tr, fe.woe_pipeline, X_logit_final.columns.tolist())
joblib.dump(bundle, "../models/transformers/preprocessing_bundle.pkl")

X.to_csv("../data/raw/X_raw_version.csv", index=False)# save 

In [ ]:
# Prepare raw X for tree
X_tree = X.copy()  # raw version

# Define your raw numeric & categorical features
tree_num_features = ["NetSpend", "GrossVolume", "TxnCount", "AvgTxnValue", "StdTxnValue", "NumUniqueProducts", "NumUniqueCategories", "NumUniqueChannels", "Recency"]  # fill with your numeric cols
tree_cat_features = ["PreferredProvider", "MostCommonPricingStrategy", "PreferredChannel", "PreferredDayOfWeek"]  # fill with your categorical cols

# Initialize
fe_tree = FeatureEngineering(
    num_features=tree_num_features,
    cat_features=tree_cat_features
)

# Build + transform
fe_tree.build_tree_pipeline()
X_tree_transformed = fe_tree.tree_pipeline.fit_transform(X_tree)

X_tree_final = pd.DataFrame(
    X_tree_transformed,
    columns=fe_tree.tree_pipeline.get_feature_names_out()
)


2025-07-02 15:47:51,342 - core.Feature_engineering - INFO - Initializing FeatureEngineering with numeric features ['NetSpend', 'GrossVolume', 'TxnCount', 'AvgTxnValue', 'StdTxnValue', 'NumUniqueProducts', 'NumUniqueCategories', 'NumUniqueChannels', 'Recency'] and categorical features ['PreferredProvider', 'MostCommonPricingStrategy', 'PreferredChannel', 'PreferredDayOfWeek']


In [ ]:
X_tree_final

,num__NetSpend,num__GrossVolume,num__TxnCount,num__AvgTxnValue,num__StdTxnValue,num__NumUniqueProducts,num__NumUniqueCategories,num__NumUniqueChannels,num__Recency,cat__PreferredProvider_ProviderId_1,...,cat__PreferredChannel_ChannelId_2,cat__PreferredChannel_ChannelId_3,cat__PreferredChannel_ChannelId_5,cat__PreferredDayOfWeek_0,cat__PreferredDayOfWeek_1,cat__PreferredDayOfWeek_2,cat__PreferredDayOfWeek_3,cat__PreferredDayOfWeek_4,cat__PreferredDayOfWeek_5,cat__PreferredDayOfWeek_6
0,-0.066891,-0.089524,-0.253459,-0.052297,-0.131508,-1.115175,-1.153977,-1.404749,1.937605,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.066891,-0.089524,-0.253459,-0.052297,-0.131508,-1.115175,-1.153977,-1.404749,1.937605,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.055849,-0.082011,-0.212186,-0.075710,-0.089197,-0.135831,-0.114953,0.450075,2.158882,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.061655,-0.091448,-0.150278,-0.109431,-0.126154,-0.135831,-0.114953,0.450075,-0.201408,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.055849,-0.081422,-0.201868,-0.080169,-0.090795,0.353841,-0.114953,0.450075,-0.717722,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,-0.055849,-0.081422,-0.201868,-0.080169,-0.089889,-0.135831,-0.114953,0.450075,-0.975879,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3738,-0.055849,-0.081422,-0.212186,-0.073798,-0.092486,-0.135831,-0.114953,0.450075,-0.201408,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3739,0.136968,0.132930,0.778355,-0.075710,0.018508,1.333185,0.924070,0.450075,-1.123397,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3740,-0.012049,-0.037600,-0.088369,-0.058972,-0.104480,-0.135831,-0.114953,0.450075,1.347532,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
X_logit_final['is_high_risk'] = y.values
X_tree_final['is_high_risk'] = y.values

X_logit_final

,StdTxnValue,NumUniqueProducts,NumUniqueCategories,NumUniqueChannels,PreferredDayOfWeek,NetSpend_woe,GrossVolume_woe,TxnCount_woe,AvgTxnValue_woe,Recency_woe,PreferredProvider_woe,MostCommonPricingStrategy_woe,PreferredChannel_woe,is_high_risk
0,-0.131508,-1.115175,-1.153977,-1.404749,-0.606251,0.755457,0.864950,0.897448,0.012640,6.892332,-0.035129,0.573687,-0.193549,1
1,-0.131508,-1.115175,-1.153977,-1.404749,-0.606251,0.755457,0.864950,0.897448,0.012640,6.892332,-0.035129,0.573687,-0.193549,1
2,-0.089197,-0.135831,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,6.892332,-0.035129,-0.119200,0.103140,1
3,-0.126154,-0.135831,-0.114953,0.450075,-0.037397,0.755457,0.864950,0.011104,0.320632,-0.981615,-0.035129,-0.119200,-0.193549,0
4,-0.090795,0.353841,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,0.000000,0.284038,-0.119200,0.103140,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,-0.089889,-0.135831,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,0.000000,0.284038,-0.119200,0.103140,0
3738,-0.092486,-0.135831,-0.114953,0.450075,0.531458,-0.006152,-0.072222,0.011104,-0.346197,-0.981615,-0.035129,-0.119200,0.103140,0
3739,0.018508,1.333185,0.924070,0.450075,0.531458,-0.952163,-0.995837,-1.413437,-0.346197,0.000000,0.284038,-0.119200,0.103140,0
3740,-0.104480,-0.135831,-0.114953,0.450075,0.531458,-0.952163,-0.995837,-1.413437,0.012640,6.892332,-0.132094,-0.119200,0.103140,1


In [ ]:
# save final data
X_logit_final.to_csv('../data/processed/logit_data.csv', index=False)
X_tree_final.to_csv('../data/processed/tree_data.csv', index=False)